In [96]:
import pandas as pd

nfl2018 = pd.read_csv("FinalData/nfl2018_final.csv")

nfl2019 = pd.read_csv("FinalData/nfl2019_final.csv")

nfl2020 = pd.read_csv("FinalData/nfl2020_final.csv")

nfl2021 = pd.read_csv("FinalData/nfl2021_final.csv")

nfl2022 = pd.read_csv("FinalData/nfl2022_final.csv")

nfl_2018_2022 = pd.concat([nfl2018, nfl2019, nfl2020, nfl2021, nfl2022]).reset_index().drop(columns=["index"])

final_data = nfl_2018_2022[nfl_2018_2022.home_win_percentage.isna() == False].reset_index().drop(columns=["index"])

In [97]:
final_data

,Week,home_team,away_team,home_win_percentage,away_win_percentage,home_points_for,away_points_for,home_points_against,away_points_against,home_yards_for,away_yards_for,home_yards_against,away_yards_against,home_to_for,away_to_for,home_to_against,away_to_against,home_win
0,2.0,Cincinnati Bengals,Baltimore Ravens,1.000000,1.000000,34.000000,47.000000,23.000000,3.000000,330.000000,369.000000,380.000000,153.000000,2.000000,1.000000,2.000000,2.000000,1
1,2.0,Atlanta Falcons,Carolina Panthers,0.000000,1.000000,12.000000,16.000000,18.000000,8.000000,299.000000,293.000000,232.000000,232.000000,1.000000,1.000000,2.000000,1.000000,1
2,2.0,Buffalo Bills,Los Angeles Chargers,0.000000,0.000000,3.000000,28.000000,47.000000,38.000000,153.000000,541.000000,369.000000,362.000000,2.000000,2.000000,1.000000,0.000000,0
3,2.0,New Orleans Saints,Cleveland Browns,0.000000,0.500000,40.000000,21.000000,48.000000,21.000000,475.000000,327.000000,529.000000,472.000000,2.000000,1.000000,0.000000,6.000000,1
4,2.0,Washington Redskins,Indianapolis Colts,1.000000,0.000000,24.000000,23.000000,6.000000,34.000000,429.000000,380.000000,213.000000,330.000000,1.000000,2.000000,2.000000,2.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,12.0,Arizona Cardinals,Los Angeles Chargers,0.363636,0.500000,21.818182,22.700000,26.909091,25.800000,331.909091,351.700000,360.272727,369.000000,1.000000,1.200000,1.272727,1.300000,0
1136,12.0,Kansas City Chiefs,Los Angeles Rams,0.800000,0.300000,30.000000,16.800000,23.300000,22.700000,429.300000,288.000000,349.200000,312.000000,1.300000,1.500000,1.000000,0.800000,1
1137,12.0,San Francisco 49ers,New Orleans Saints,0.600000,0.363636,23.600000,22.636364,17.300000,24.272727,362.700000,355.181818,283.900000,327.272727,1.300000,1.727273,1.200000,0.636364,1
1138,12.0,Philadelphia Eagles,Green Bay Packers,0.900000,0.363636,26.300000,18.363636,18.300000,22.090909,370.600000,344.909091,300.600000,328.272727,0.900000,1.363636,2.000000,1.000000,1


In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

In [111]:
#credit to GitHub user MatthewR1021: https://github.com/MatthewR1021/MLB_Classification

def new_acc(probabilities):
    class_list2=[]
    for log in probabilities:
        if log[0]<.40:
            class_list2.append(1)
        elif log[0]>.60:
            class_list2.append(0)
        else:
            class_list2.append(3)
    y_pred_class=pd.DataFrame(class_list2)
    y_pred_class.reset_index(inplace=True)
    y_test_df=pd.DataFrame(y_test)
    y_test_df.reset_index(inplace=True)
    comb=pd.concat([y_test_df,y_pred_class],axis=1)
    comb.drop(['index','index'],axis=1,inplace=True)
    comb['pred']=comb[0]
    comb.drop([0],axis=1,inplace=True)
    comb.drop(comb[comb['pred'] == 3].index, inplace = True)
    count=0
    for bools in comb['home_win']==comb['pred']:
        if bools is True:
            count+=1
        else:
            pass
    accuracy=count/len(comb)
    
    return f'New Accuracy: {accuracy}',f'Amount of Games: {count}'

In [112]:
X = final_data.drop(columns=["Week", "home_team", "away_team", "home_win"])
y = final_data["home_win"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [120]:
log_reg = LogisticRegression()

log_reg.fit(X_train_scaled, y_train)

log_reg_predictions = log_reg.predict(X_test_scaled)

log_reg_probabilities = log_reg.predict_proba(X_test_scaled)

log_reg_score = accuracy_score(y_test, log_reg_predictions)

print("Logistic Regression Accuracy: " + str(log_reg_score) + ", Amount of Games: " + str(len(y_test)))

Logistic Regression Accuracy: 0.6198830409356725, Amount of Games: 171


In [121]:
#only chooses to "bet" on games where one of the teams has a 60% or higher chance of winning
print(new_acc(log_reg_probabilities))

('New Accuracy: 0.7073170731707317', 'Amount of Games: 58')


In [105]:
xgb = XGBClassifier()

xgb.fit(X_train_scaled, y_train)

xgb_predictions = xgb.predict(X_test_scaled)

xgb_score = accuracy_score(y_test, xgb_predictions)

print("XGBoost Accuracy: " + str(xgb_score))

XGBoost Accuracy: 0.5146198830409356


In [125]:
rf = RandomForestClassifier(n_estimators=100)

rf.fit(X_train_scaled, y_train)

rf_predictions = rf.predict(X_test_scaled)

rf_score = accuracy_score(y_test, rf_predictions)

print("Random Forest Accuracy: " + str(rf_score))

Random Forest Accuracy: 0.5906432748538012
